## Scraping

In [18]:
!pip install requests beautifulsoup4 pandas matplotlib
!pip install requests-html
!pip install lxml_html_clean
!pip install "lxml[html_clean]"
!pip install playwright
!playwright install
!pip install tabulate



  Using cached matplotlib-3.9.4-cp39-cp39-macosx_11_0_arm64.whl (7.8 MB)
  Using cached contourpy-1.3.0-cp39-cp39-macosx_11_0_arm64.whl (249 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-macosx_11_0_arm64.whl (64 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
  Using cached fonttools-4.58.1-cp39-cp39-macosx_10_9_universal2.whl (2.7 MB)
You should consider upgrading via the '/Users/macbook/Documents/Scraping/env/bin/python3 -m pip install --upgrade pip' command.
  Using cached requests_html-0.10.0-py3-none-any.whl (13 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
  Using cached w3lib-2.3.1-py3-none-any.whl (21 kB)
  Using cached fake_useragent-2.2.0-py3-none-any.whl (161 kB)
  Using cached pyppeteer-2.0.0-py3-none-any.whl (82 kB)
  Using cached pyquery-2.0.1-py3-none-any.whl (22 kB)
  Using cached parse-1.20.2-py2.py3-none-any.whl (20 kB)
  

In [20]:
import requests
from bs4 import BeautifulSoup

url = "https://www.sebi.gov.in/sebiweb/other/OtherAction.do?doPmr=yes"

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.content, "html.parser")

select_pm = soup.find("select", {"name": "pmrId"})
if not select_pm:
    raise Exception("Portfolio Manager select box not found")

options = [(opt.get("value"), opt.text.strip()) for opt in select_pm.find_all("option") if opt.get("value")]

print("Portfolio Manager options (value, text):")
for value, text in options:
    print(f"{value} -> {text}")


Portfolio Manager options (value, text):
INP000008464@@INP000008464@@1729 ADVISORS LLP -> 1729 ADVISORS LLP
INP000005190@@INP000005190@@2POINT2 CAPITAL ADVISORS LLP -> 2POINT2 CAPITAL ADVISORS LLP
INP000008640@@INP000008640@@360 ONE ALTERNATES ASSET MANAGEMENT LIMITED -> 360 ONE ALTERNATES ASSET MANAGEMENT LIMITED
INP000004565@@INP000004565@@360 ONE ASSET MANAGEMENT LIMITED -> 360 ONE ASSET MANAGEMENT LIMITED
INP000005874@@INP000005874@@360 ONE PORTFOLIO MANAGERS LIMITED -> 360 ONE PORTFOLIO MANAGERS LIMITED
INP000006679@@INP000006679@@4A SECURITIES -> 4A SECURITIES
INP000004730@@INP000004730@@A C CHOKSI SHARE BROKERS PRIVATE LIMITED -> A C CHOKSI SHARE BROKERS PRIVATE LIMITED
INP000003674@@INP000003674@@A. K. WEALTH MANAGEMENT PRIVATE LIMITED -> A. K. WEALTH MANAGEMENT PRIVATE LIMITED
INP000004599@@INP000004599@@AAROHAN HOLDINGS AND ADVISORS PRIVATE LIMITED -> AAROHAN HOLDINGS AND ADVISORS PRIVATE LIMITED
INP000008224@@INP000008224@@AAVISHKAAR ADVISORS PRIVATE LIMITED -> AAVISHKAAR AD

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import calendar

# def fetch_tables_with_titles(pmr_id, year, month):
#     """
#     Fetch tables with their titles from the PMR page for given year and month.
#     Returns a dict: { "XX - Table Title": [ [row1], [row2], ... ] }
#     """
#     url = "https://www.sebi.gov.in/sebiweb/other/OtherAction.do?doPmr=yes"

#     headers = {
#         "User-Agent": "Mozilla/5.0",
#         "Referer": url,
#         "Origin": "https://www.sebi.gov.in"
#     }

#     payload = {
#         "pmrId": pmr_id,
#         "year": str(year),
#         "month": str(month),
#         "action": "search"
#     }

#     tables_data = {}

#     with requests.Session() as session:
#         session.headers.update(headers)
#         session.get(url)

#         response = session.post(url, data=payload)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, "html.parser")

#         all_elements = soup.find_all(['strong', 'b', 'table'])

#         current_title = "Untitled Section"
#         table_count = 0

#         for el in all_elements:
#             if el.name in ['strong', 'b']:
#                 title_text = el.get_text(strip=True)
#                 if title_text:
#                     current_title = title_text
#             elif el.name == 'table' and 'statistics-table' in el.get('class', []):
#                 table_count += 1
#                 rows = []
#                 for row in el.find_all("tr"):
#                     cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
#                     if cols:
#                         rows.append(cols)
#                 tables_data[f"{table_count:02d} - {current_title}"] = rows

#     return tables_data

import requests
from bs4 import BeautifulSoup

def fetch_tables_with_titles_no_session(pmr_id, year, month):
    url = "https://www.sebi.gov.in/sebiweb/other/OtherAction.do?doPmr=yes"

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": url,
        "Origin": "https://www.sebi.gov.in"
    }

    payload = {
        "pmrId": pmr_id,
        "year": str(year),
        "month": str(month),
        "action": "search"
    }

    # First: GET request to fetch cookies (and possibly CSRF token)
    get_response = requests.get(url, headers=headers)
    get_response.raise_for_status()

    # Extract cookies from the GET response
    cookies = get_response.cookies

    # Second: POST request with the same headers and cookies
    post_response = requests.post(url, data=payload, headers=headers, cookies=cookies)
    post_response.raise_for_status()

    soup = BeautifulSoup(post_response.content, "html.parser")

    all_elements = soup.find_all(['strong', 'b', 'table'])
    current_title = "Untitled Section"
    table_count = 0
    tables_data = {}

    for el in all_elements:
        if el.name in ['strong', 'b']:
            title_text = el.get_text(strip=True)
            if title_text:
                current_title = title_text
        elif el.name == 'table' and 'statistics-table' in el.get('class', []):
            table_count += 1
            rows = []
            for row in el.find_all("tr"):
                cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                if cols:
                    rows.append(cols)
            tables_data[f"{table_count:02d} - {current_title}"] = rows

    return tables_data



def extract_funds_inflow_outflow_table(tables_dict):
    target_key = None
    for key in tables_dict.keys():
        if "07 - C.Funds Inflow/ Outflow" in key:
            target_key = key
            break
    
    if target_key is None:
        return None  # Table not found
    
    data = tables_dict[target_key]

    # Try to find the header row with max columns
    max_cols = max(len(row) for row in data)
    
    # Find a header row with max columns (instead of just first row)
    header_row_index = 0
    for i, row in enumerate(data):
        if len(row) == max_cols:
            header_row_index = i
            break
    
    columns = data[header_row_index]
    # Data rows are everything after header row
    df = pd.DataFrame(data[header_row_index+1:], columns=columns)
    return df



def fetch_monthly_funds_data(pmr_id, months_list):
    """
    Fetch "07 - C.Funds Inflow/ Outflow" table data for multiple months.
    months_list: list of (year, month) tuples.
    Returns dict { "Month Year": DataFrame }
    """
    all_data = {}

    for year, month in months_list:
        month_name = calendar.month_name[month]
        month_year = f"{month_name} {year}"
        print(f"Fetching data for {month_year}...")
        tables = fetch_tables_with_titles(pmr_id, year, month)
        df = extract_funds_inflow_outflow_table(tables)
        if df is not None:
            all_data[month_year] = df
        else:
            print(f"Warning: '07 - C.Funds Inflow/ Outflow' table not found for {month_year}")

    print("Fetching complete.")
    return all_data


# Example usage:

from tabulate import tabulate

if __name__ == "__main__":
    pmr_id = "INP000008464@@INP000008464@@1729 ADVISORS LLP"
    all_months = [(2024, m) for m in range(4, 13)] + [(2025, m) for m in range(1, 4)]

    monthly_funds_data = fetch_monthly_funds_data(pmr_id, all_months)

    for month_year, df in monthly_funds_data.items():
        print(f"\nData for {month_year}:")
        print(f"Columns: {df.columns.tolist()}\n")
        # Print a nicely formatted table of the first 5 rows
        print(tabulate(df.head(), headers='keys', tablefmt='fancy_grid', showindex=False))




Fetching data for April 2024...
Fetching data for May 2024...
Fetching data for June 2024...
Fetching data for July 2024...
Fetching data for August 2024...
Fetching data for September 2024...
Fetching data for October 2024...
Fetching data for November 2024...
Fetching data for December 2024...
Fetching data for January 2025...
Fetching data for February 2025...
Fetching data for March 2025...
Fetching complete.

Data for April 2024:
Columns: ['', 'Inflow during the month(in INR crores)', 'Outflow during the month(in INR crores)', 'Net Inflow (+ve)/ Outflow (-ve) during the month(in INR crores)', 'Inflow during the FY since April 01 to April 2024(in INR crores)', 'Outflow during the FY since April 01 to  April 2024(in INR crores)', 'Net Inflow (+ve)/ Outflow (-ve) during the FY since April 01 to  April 2024(in INR crores)']

╒══════════════════════════════════╤══════════════════════════════════════════╤═══════════════════════════════════════════╤═══════════════════════════════════════

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import calendar
import matplotlib.pyplot as plt

def fetch_only_funds_inflow_outflow(pmr_id, year, month):
    import requests
    from bs4 import BeautifulSoup

    url = "https://www.sebi.gov.in/sebiweb/other/OtherAction.do?doPmr=yes"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": url,
        "Origin": "https://www.sebi.gov.in"
    }
    payload = {
        "pmrId": pmr_id,
        "year": str(year),
        "month": str(month),
        "action": "search"
    }

    # 1. Make initial GET request to get cookies
    get_response = requests.get(url, headers=headers)
    get_response.raise_for_status()
    cookies = get_response.cookies

    # 2. Make POST request with cookies from GET and same headers
    post_response = requests.post(url, data=payload, headers=headers, cookies=cookies)
    post_response.raise_for_status()

    soup = BeautifulSoup(post_response.content, "html.parser")
    all_elements = soup.find_all(['strong', 'b', 'table'])

    current_title = "Untitled Section"
    table_count = 0

    for el in all_elements:
        if el.name in ['strong', 'b']:
            title_text = el.get_text(strip=True)
            if title_text:
                current_title = title_text
        elif el.name == 'table' and 'statistics-table' in el.get('class', []):
            table_count += 1
            full_title = f"{table_count:02d} - {current_title}"
            if full_title.strip() == "07 - C.Funds Inflow/ Outflow":
                rows = []
                for row in el.find_all("tr"):
                    cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                    if cols:
                        rows.append(cols)
                return {full_title: rows}

    return {}  # If not found






def extract_funds_inflow_outflow_table(tables_dict):
    target_key = None
    for key in tables_dict.keys():
        if "07 - C.Funds Inflow/ Outflow" in key:
            target_key = key
            break
    if target_key is None:
        return None
    data = tables_dict[target_key]
    print("data : ",data)
    max_cols = max(len(row) for row in data)
    header_row_index = 0
    for i, row in enumerate(data):
        if len(row) == max_cols:
            header_row_index = i
            break
    columns = data[header_row_index]
    print("columns : ",columns)
    df = pd.DataFrame(data[header_row_index+1:], columns=columns)
    return df


def fetch_monthly_funds_data(pmr_id, months_list):
    all_data = {}
    for year, month in months_list:
        month_name = calendar.month_name[month]
        month_year = f"{month_name} {year}"
        print(f"Fetching data for {month_year}...")
        tables = fetch_only_funds_inflow_outflow(pmr_id, year, month)
        df = extract_funds_inflow_outflow_table(tables)
        if df is not None:
            all_data[month_year] = df
        else:
            print(f"Warning: '07 - C.Funds Inflow/ Outflow' table not found for {month_year}")
    print("Fetching complete.")
    return all_data


def prepare_performance_data(monthly_funds_data):
    months = []
    monthly_net_flow = []
    cumulative_net_flow = []

    for month_year in sorted(monthly_funds_data.keys(), key=lambda x: pd.to_datetime(x, format='%B %Y')):
        df = monthly_funds_data[month_year]
        df.columns = [col.strip() for col in df.columns]
        print(f"Processing {month_year}...")
        print("Columns:", df.columns.tolist())
        
        total_row = df[df[''].str.lower() == 'total']
        if total_row.empty:
            print(f"Warning: Total row not found for {month_year}, skipping.")
            continue

        # Look for columns containing the needed text ignoring FY end date variance
        monthly_col = None
        cumulative_col = None
        for col in df.columns:
            if "Net Inflow (+ve)/ Outflow (-ve) during the month" in col:
                monthly_col = col
            if "Net Inflow (+ve)/ Outflow (-ve) during the FY since April 01 to" in col:
                cumulative_col = col

        if not monthly_col or not cumulative_col:
            print(f"Warning: Required columns not found for {month_year}, skipping.")
            continue

        try:
            monthly_net = float(total_row.iloc[0][monthly_col].replace(',', ''))
            cumulative_net = float(total_row.iloc[0][cumulative_col].replace(',', ''))
        except Exception as e:
            print(f"Error parsing numbers for {month_year}: {e}, skipping.")
            continue
        
        print(f"Extracted values - Monthly: {monthly_net}, Cumulative: {cumulative_net}")

        months.append(month_year)
        monthly_net_flow.append(monthly_net)
        cumulative_net_flow.append(cumulative_net)

    print("Final extracted months:", months)
    return months, monthly_net_flow, cumulative_net_flow



def plot_performance_chart(months, monthly_net_flow, cumulative_net_flow):
    plt.figure(figsize=(12,6))
    plt.plot(months, monthly_net_flow, marker='o', label='Monthly Net Inflow/Outflow')
    plt.plot(months, cumulative_net_flow, marker='s', label='Cumulative Net Inflow/Outflow FY')
    plt.xticks(rotation=45)
    plt.title("Funds Inflow/Outflow Performance (Apr 2024 - Mar 2025)")
    plt.xlabel("Month")
    plt.ylabel("Amount (INR crores)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    pmr_id = "INP000008464@@INP000008464@@1729 ADVISORS LLP"
    all_months = [(2024, m) for m in range(4, 13)] + [(2025, m) for m in range(1, 4)]
    print("all_months : ",all_months)
    monthly_funds_data = fetch_monthly_funds_data(pmr_id, all_months)
    # print("monthly_funds_data : ",monthly_funds_data)

    months, monthly_net_flow, cumulative_net_flow = prepare_performance_data(monthly_funds_data)
    print("months, monthly_net_flow, cumulative_net_flow : ",months, monthly_net_flow, cumulative_net_flow )
    plot_performance_chart(months, monthly_net_flow, cumulative_net_flow)


all_months :  [(2024, 4), (2024, 5), (2024, 6), (2024, 7), (2024, 8), (2024, 9), (2024, 10), (2024, 11), (2024, 12), (2025, 1), (2025, 2), (2025, 3)]
